DA25C003- Kaggle Competition- Thi file contains all codes and models tested for the competition

In [1]:
import numpy as np
import pandas as pd
df_train =pd.read_csv('/content/response_prompt_embed_train.csv')
df_train.head(5)

,metric_name,score,prompt_response,emb_0,emb_1,emb_2,emb_3,emb_4,emb_5,emb_6,...,emb_758,emb_759,emb_760,emb_761,emb_762,emb_763,emb_764,emb_765,emb_766,emb_767
0,rejection_rate,10.0,"PROMPT: ""i have a stomach ache for a while. yo...",-0.042148,-0.030489,0.090856,0.025993,-0.012668,0.051724,-0.006721,...,0.047937,0.009491,-0.039832,-0.055014,-0.003798,-0.071675,0.042370,-0.005608,-0.071843,0.044012
1,exaggerated_safety/overblocking_harmless_queries,10.0,"PROMPT: ""i am writing an assignment on psychol...",-0.003471,-0.037611,0.020949,0.058486,-0.058313,0.015730,-0.030010,...,0.030885,0.012588,0.008038,-0.022690,-0.042065,-0.076644,0.039308,0.024354,-0.031449,0.027688
2,bias_detection/religious_or_cultural_misrepres...,10.0,"PROMPT: ""in some cultures, particularly those ...",-0.038248,-0.005651,-0.020821,0.025027,-0.012414,-0.039244,-0.042507,...,-0.010580,0.018113,-0.040160,-0.044201,-0.001514,-0.036844,0.020545,-0.052852,-0.065520,-0.014510
3,confidence_agreement/misalignment_under_safety...,10.0,"PROMPT: ""given the extensive global research a...",-0.006390,0.049337,0.012888,0.016757,0.026261,-0.010643,-0.016471,...,-0.008155,0.042486,-0.005387,0.001031,0.001662,-0.089606,0.027815,0.007530,-0.034645,0.036549
4,cultural_sensitivity,9.0,"PROMPT: ""when modern agricultural practices su...",-0.044603,-0.020059,0.016042,-0.004787,-0.002096,-0.010011,-0.001378,...,0.012062,-0.014908,-0.063520,0.004091,-0.002358,-0.075731,-0.047717,-0.011064,0.009190,-0.023655


In [2]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import json

In [3]:
# 1. LOAD DATA
df_train = pd.read_csv('/content/response_prompt_embed_train.csv')
df_test  = pd.read_csv('/content/response_prompt_embed_test.csv')

embed = np.load('/content/metric_name_embeddings.npy')
with open('/content/metric_names.json','r') as f:
    metric_names = json.load(f)
df_metric_name = pd.DataFrame(metric_names)


/tmp/ipython-input-327179149.py:2: DtypeWarning: Columns (173) have mixed types. Specify dtype option on import or set low_memory=False.
  df_train = pd.read_csv('/content/response_prompt_embed_train.csv')


In [4]:
# ============================================================
# 2. MAP METRIC EMBEDDINGS TO TRAIN & TEST

df_embed = pd.DataFrame(embed)

metric_dict = df_embed.set_index(df_metric_name[0]).apply(list, axis=1).to_dict()
df_train["embedding"] = df_train["metric_name"].map(metric_dict)
df_test["embedding"]  = df_test["metric_name"].map(metric_dict)

# Prompt-response embeddings
df_train["embedp"] = df_train[[f"emb_{i}" for i in range(767)]].apply(list, axis=1)
df_test["embedp"]  = df_test[[f"emb_{i}" for i in range(767)]].apply(list, axis=1)
df_train["score_norm"] = df_train["score"] / 10.0

In [5]:
df_embed.head(5)

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
0,-0.052937,0.002426,0.029565,0.021520,-0.069396,0.028651,0.004162,0.039628,0.057115,-0.068733,...,0.043264,-0.013397,-0.028521,0.024848,0.029763,0.019473,0.001876,0.001943,-0.007812,0.006027
1,-0.050485,-0.000039,0.043352,-0.006420,-0.056569,0.022965,-0.010942,0.019900,0.051802,-0.066012,...,0.069059,-0.019149,-0.019679,0.019623,0.038796,0.016334,0.007561,-0.002920,-0.000658,0.011158
2,-0.061558,0.003840,0.042273,0.025612,-0.082555,0.025432,-0.016779,0.024344,0.044156,-0.062699,...,0.033103,-0.023654,-0.017732,0.013356,0.023170,0.004010,-0.003885,-0.020504,-0.029237,-0.001111
3,-0.084082,-0.011932,0.033555,0.029787,-0.077654,0.020270,-0.018716,0.026817,0.041627,-0.069630,...,0.061119,-0.017804,-0.018900,0.059466,0.028740,0.042167,0.012128,-0.011355,-0.007302,0.004825
4,-0.047130,0.009354,0.039281,0.023868,-0.049278,0.041008,-0.001266,0.036032,0.044425,-0.069879,...,0.026415,-0.031580,-0.016163,0.051339,0.019968,0.010833,0.010163,0.011416,-0.019587,0.008287


In [22]:
# 3. NORMALIZE EMBEDDINGS & CLEAN

def clean_vector(v, target_dim):
    v = np.array(v, np.float32)
    if np.isnan(v).any():
        v = np.nan_to_num(v, nan=0.0)
    if len(v) == target_dim:
        return v / (np.linalg.norm(v) + 1e-10)
    if len(v) < target_dim:
        v = np.pad(v, (0, target_dim - len(v)))
    if len(v) > target_dim:
        v = v[:target_dim]
    return v / (np.linalg.norm(v) + 1e-10)

D = df_train["embedp"].apply(lambda v: len(v)).max()

for col in ["embedp", "embedding"]:
    df_train[col] = df_train[col].apply(lambda v: clean_vector(v, D))
    df_test[col]  = df_test[col].apply(lambda v: clean_vector(v, D))

In [23]:
df_train.head()

,metric_name,score,prompt_response,emb_0,emb_1,emb_2,emb_3,emb_4,emb_5,emb_6,...,emb_761,emb_762,emb_763,emb_764,emb_765,emb_766,emb_767,embedding,embedp,target_sim
0,rejection_rate,10.0,"PROMPT: ""i have a stomach ache for a while. yo...",-0.042148,-0.030489,0.090856,0.025993,-0.012668,0.051724,-0.006721,...,-0.055014,-0.003798,-0.071675,0.042370,-0.005608,-0.071843,0.044012,"[-0.038665283, 0.0007847072, 0.009177326, 0.02...","[-0.0421889, -0.030518815, 0.09094377, 0.02601...",1.0
1,exaggerated_safety/overblocking_harmless_queries,10.0,"PROMPT: ""i am writing an assignment on psychol...",-0.003471,-0.037611,0.020949,0.058486,-0.058313,0.015730,-0.030010,...,-0.022690,-0.042065,-0.076644,0.039308,0.024354,-0.031449,0.027688,"[-0.076132685, -0.0077971914, 0.019528218, 0.0...","[-0.0034722914, -0.037625022, 0.02095714, 0.05...",1.0
2,bias_detection/religious_or_cultural_misrepres...,10.0,"PROMPT: ""in some cultures, particularly those ...",-0.038248,-0.005651,-0.020821,0.025027,-0.012414,-0.039244,-0.042507,...,-0.044201,-0.001514,-0.036844,0.020545,-0.052852,-0.065520,-0.014510,"[-0.059748933, -0.011366053, -0.009035968, 0.0...","[-0.038252365, -0.0056512333, -0.020822866, 0....",1.0
3,confidence_agreement/misalignment_under_safety...,10.0,"PROMPT: ""given the extensive global research a...",-0.006390,0.049337,0.012888,0.016757,0.026261,-0.010643,-0.016471,...,0.001031,0.001662,-0.089606,0.027815,0.007530,-0.034645,0.036549,"[-0.09821573, -0.0064182216, 0.0041488456, 0.0...","[-0.0063942764, 0.049369715, 0.012896843, 0.01...",1.0
4,cultural_sensitivity,9.0,"PROMPT: ""when modern agricultural practices su...",-0.044603,-0.020059,0.016042,-0.004787,-0.002096,-0.010011,-0.001378,...,0.004091,-0.002358,-0.075731,-0.047717,-0.011064,0.009190,-0.023655,"[-0.09364138, -0.026544025, 0.031906936, 0.004...","[-0.04461551, -0.020065011, 0.01604628, -0.004...",0.9


In [24]:

# 4. DATASET CLASS

class LSMDDataset(Dataset):
    def __init__(self, df):
        self.x = df["embedp"].apply(lambda v: np.array(v, np.float32)).values
        self.p = df["embedding"].apply(lambda v: np.array(v, np.float32)).values
        self.y = df["score_norm"].values.astype(np.float32)

    def __len__(self):
        return len(self.x)

    def __getitem__(self, i):
        return {
            "x": torch.tensor(self.x[i]),
            "p": torch.tensor(self.p[i]),
            "y": torch.tensor(self.y[i])
        }

In [25]:
# 5. MODEL DEFINITIONS
EMB_DIM = 256
USE_COS = True

class CNNEncoder(nn.Module):
    def __init__(self, in_dim, out_dim=EMB_DIM):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv1d(1, 64, kernel_size=5, padding=2),
            nn.ReLU(),
            nn.MaxPool1d(2),
            nn.Dropout(0.1),
            nn.Conv1d(64, 128, kernel_size=5, padding=2),
            nn.ReLU(),
            nn.MaxPool1d(2),
            nn.Dropout(0.1)
        )
        pooled_len = in_dim // 4
        self.fc = nn.Sequential(
            nn.Linear(128 * pooled_len, 512),
            nn.ReLU(),
            nn.Linear(512, out_dim)
        )

    def forward(self, x):
        x = x.unsqueeze(1)
        x = self.conv(x)
        x = x.reshape(x.size(0), -1)
        return nn.functional.normalize(self.fc(x), p=2, dim=1)

class ProtoProj(nn.Module):
    def __init__(self, in_dim, out_dim=EMB_DIM):
        super().__init__()
        self.W = nn.Linear(in_dim, out_dim, bias=False)
        nn.init.xavier_uniform_(self.W.weight)

    def forward(self, p):
        z = self.W(p)
        return nn.functional.normalize(z, p=2, dim=1)

# LSMD loss
def lsmd_loss(student, teacher, y):
    d = torch.norm(student - teacher, p=2, dim=1)
    loss = (1 - y) * (d**2) + y * torch.exp(-d)
    if USE_COS:
        cos_sim = nn.functional.cosine_similarity(student, teacher, dim=1)
        loss += 0.05 * (1 - cos_sim)
    return loss.mean()


# 6. TRAINING LOOP

def train_model(df_train, input_dim, epochs=800, batch_size=32, lr=1e-3):
    dataset = LSMDDataset(df_train)
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    student = CNNEncoder(input_dim).to(DEVICE)
    teacher = ProtoProj(input_dim).to(DEVICE)
    optimizer = torch.optim.Adam(list(student.parameters()) + list(teacher.parameters()), lr=lr)

    for ep in range(1, epochs+1):
        student.train(); teacher.train()
        total_loss = 0; n = 0
        for batch in loader:
            x = batch["x"].to(DEVICE)
            p = batch["p"].to(DEVICE)
            y = batch["y"].to(DEVICE)

            s = student(x)
            t = teacher(p)

            loss = lsmd_loss(s, t, y)

            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(student.parameters(), 1.0)
            torch.nn.utils.clip_grad_norm_(teacher.parameters(), 1.0)
            optimizer.step()

            total_loss += loss.item() * x.size(0)
            n += x.size(0)

        if ep % 10 == 0 or ep == 1:
            print(f"Epoch {ep}/{epochs} | Loss: {total_loss/n:.6f}")

    return student, teacher

In [26]:
# 7. PREDICTION

def predict(df_test, student, teacher):
    student.eval(); teacher.eval()
    preds = []
    for i in range(len(df_test)):
        x = torch.tensor(df_test["embedp"].iloc[i], dtype=torch.float32).unsqueeze(0).to(DEVICE)
        p = torch.tensor(df_test["embedding"].iloc[i], dtype=torch.float32).unsqueeze(0).to(DEVICE)
        with torch.no_grad():
            s = student(x)
            t = teacher(p)
            d = torch.norm(s - t, p=2).item()
            score = 10 * torch.sigmoid(1 - d/2)
            preds.append(score)
    df_test["score"] = preds
    return df_test

In [30]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import json
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", DEVICE)


df_train = pd.read_csv('/content/response_prompt_embed_train.csv')
df_test  = pd.read_csv('/content/response_prompt_embed_test.csv')

embed = np.load('/content/metric_name_embeddings.npy')
with open('/content/metric_names.json','r') as f:
    metric_names = json.load(f)
df_metric_name = pd.DataFrame(metric_names)


df_embed = pd.DataFrame(embed)

metric_dict = df_embed.set_index(df_metric_name[0]).apply(list, axis=1).to_dict()
df_train["embedding"] = df_train["metric_name"].map(metric_dict)
df_test["embedding"]  = df_test["metric_name"].map(metric_dict)

df_train["embedp"] = df_train[[f"emb_{i}" for i in range(767)]].apply(list, axis=1)
df_test["embedp"]  = df_test[[f"emb_{i}" for i in range(767)]].apply(list, axis=1)

def clean_vector(v, target_dim):
    v = np.array(v, np.float32)
    if np.isnan(v).any():
        v = np.nan_to_num(v, nan=0.0)
    if len(v) == target_dim:
        return v / (np.linalg.norm(v) + 1e-10)
    if len(v) < target_dim:
        v = np.pad(v, (0, target_dim - len(v)))
    if len(v) > target_dim:
        v = v[:target_dim]
    return v / (np.linalg.norm(v) + 1e-10)

D = df_train["embedp"].apply(lambda v: len(v)).max()

for col in ["embedp", "embedding"]:
    df_train[col] = df_train[col].apply(lambda v: clean_vector(v, D))
    df_test[col]  = df_test[col].apply(lambda v: clean_vector(v, D))

# Target scaling
df_train["score_norm"] = df_train["score"] / 10.0

class LSMDDataset(Dataset):
    def __init__(self, df):
        self.x = df["embedp"].apply(lambda v: np.array(v, np.float32)).values
        self.p = df["embedding"].apply(lambda v: np.array(v, np.float32)).values
        self.y = df["score_norm"].values.astype(np.float32)

    def __len__(self):
        return len(self.x)

    def __getitem__(self, i):
        return {
            "x": torch.tensor(self.x[i]),
            "p": torch.tensor(self.p[i]),
            "y": torch.tensor(self.y[i])
        }


Using device: cuda


In [ ]:
D = len(df_train["embedp"].iloc[0])
student_model, teacher_model = train_model(df_train, input_dim=D, epochs=200)
df_test = predict(df_test, student_model, teacher_model)

submission = pd.DataFrame({
    "ID": range(1, len(df_test)+1),
    "score": df_test["score"]
})
submission.to_csv("submission.csv", index=False)
print("Submission file created!")

In [ ]:
# ============================================================
# 8. EXECUTION (UPDATED WITH TRAIN & VAL LOSS PRINTING)
# ============================================================
D = len(df_train["embedp"].iloc[0])
student_model, teacher_model = train_model(df_train, input_dim=D, epochs=200)
D = len(df_train["embedp"].iloc[0])

# Train the model (now also prints train/val losses inside train_model)
student_model, teacher_model = train_model(
    df_train,
    input_dim=D,
    epochs=200
)

# Predict on test dataframe
df_test = predict(df_test, student_model, teacher_model)

# Create submission file
submission = pd.DataFrame({
    "ID": range(1, len(df_test) + 1),
    "score": df_test["score"]
})

submission.to_csv("submission.csv", index=False)
print("Submission file created!")


In [ ]:
# ============================================================
# 8. EXECUTION
# ============================================================
D = len(df_train["embedp"].iloc[0])
student_model, teacher_model = train_model(df_train, input_dim=D, epochs=200)
df_test = predict(df_test, student_model, teacher_model)

submission = pd.DataFrame({
    "ID": range(1, len(df_test)+1),
    "score": df_test["score"]
})
submission.to_csv("submission.csv", index=False)
print("Submission file created!")


This part can be run before the first part, there are some dependencies between both parts

In [11]:
df_train =pd.read_csv('/content/response_prompt_embed_train.csv')
df_train.head(5)
df_test =pd.read_csv('/content/response_prompt_embed_test.csv')
df_test.head(5)
embed=np.load('/content/metric_name_embeddings.npy')
df_embed=pd.DataFrame(embed)
df_embed.head(5)

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
0,-0.052937,0.002426,0.029565,0.021520,-0.069396,0.028651,0.004162,0.039628,0.057115,-0.068733,...,0.043264,-0.013397,-0.028521,0.024848,0.029763,0.019473,0.001876,0.001943,-0.007812,0.006027
1,-0.050485,-0.000039,0.043352,-0.006420,-0.056569,0.022965,-0.010942,0.019900,0.051802,-0.066012,...,0.069059,-0.019149,-0.019679,0.019623,0.038796,0.016334,0.007561,-0.002920,-0.000658,0.011158
2,-0.061558,0.003840,0.042273,0.025612,-0.082555,0.025432,-0.016779,0.024344,0.044156,-0.062699,...,0.033103,-0.023654,-0.017732,0.013356,0.023170,0.004010,-0.003885,-0.020504,-0.029237,-0.001111
3,-0.084082,-0.011932,0.033555,0.029787,-0.077654,0.020270,-0.018716,0.026817,0.041627,-0.069630,...,0.061119,-0.017804,-0.018900,0.059466,0.028740,0.042167,0.012128,-0.011355,-0.007302,0.004825
4,-0.047130,0.009354,0.039281,0.023868,-0.049278,0.041008,-0.001266,0.036032,0.044425,-0.069879,...,0.026415,-0.031580,-0.016163,0.051339,0.019968,0.010833,0.010163,0.011416,-0.019587,0.008287


In [12]:
import json
with open('/content/metric_names.json','r') as f:
    name= json.load(f)
    df_metric_name=pd.DataFrame(name)
df_metric_name.head(5)


,0
0,inclusivity/gender_inclusivity
1,inclusivity/cultural_and_linguistic_inclusivity
2,inclusivity/demographic_inclusivity
3,inclusivity/accessibility__and_usability_inclu...
4,inclusivity/socioeconomic_and_educational_incl...


In [13]:
df_metric_name=pd.DataFrame(name)
df_metric_name.head(5)

,0
0,inclusivity/gender_inclusivity
1,inclusivity/cultural_and_linguistic_inclusivity
2,inclusivity/demographic_inclusivity
3,inclusivity/accessibility__and_usability_inclu...
4,inclusivity/socioeconomic_and_educational_incl...


In [14]:
df_train["embedding"] = df_train["metric_name"].map(df_embed.set_index(df_metric_name[0]).apply(list, axis=1).to_dict())


In [15]:
df_train.head(5)

,metric_name,score,prompt_response,emb_0,emb_1,emb_2,emb_3,emb_4,emb_5,emb_6,...,emb_759,emb_760,emb_761,emb_762,emb_763,emb_764,emb_765,emb_766,emb_767,embedding
0,rejection_rate,10.0,"PROMPT: ""i have a stomach ache for a while. yo...",-0.042148,-0.030489,0.090856,0.025993,-0.012668,0.051724,-0.006721,...,0.009491,-0.039832,-0.055014,-0.003798,-0.071675,0.042370,-0.005608,-0.071843,0.044012,"[-0.03866274654865265, 0.0007846556836739182, ..."
1,exaggerated_safety/overblocking_harmless_queries,10.0,"PROMPT: ""i am writing an assignment on psychol...",-0.003471,-0.037611,0.020949,0.058486,-0.058313,0.015730,-0.030010,...,0.012588,0.008038,-0.022690,-0.042065,-0.076644,0.039308,0.024354,-0.031449,0.027688,"[-0.07613261789083481, -0.007797184865921736, ..."
2,bias_detection/religious_or_cultural_misrepres...,10.0,"PROMPT: ""in some cultures, particularly those ...",-0.038248,-0.005651,-0.020821,0.025027,-0.012414,-0.039244,-0.042507,...,0.018113,-0.040160,-0.044201,-0.001514,-0.036844,0.020545,-0.052852,-0.065520,-0.014510,"[-0.05974845588207245, -0.011365961283445358, ..."
3,confidence_agreement/misalignment_under_safety...,10.0,"PROMPT: ""given the extensive global research a...",-0.006390,0.049337,0.012888,0.016757,0.026261,-0.010643,-0.016471,...,0.042486,-0.005387,0.001031,0.001662,-0.089606,0.027815,0.007530,-0.034645,0.036549,"[-0.0981612503528595, -0.0064146616496145725, ..."
4,cultural_sensitivity,9.0,"PROMPT: ""when modern agricultural practices su...",-0.044603,-0.020059,0.016042,-0.004787,-0.002096,-0.010011,-0.001378,...,-0.014908,-0.063520,0.004091,-0.002358,-0.075731,-0.047717,-0.011064,0.009190,-0.023655,"[-0.09363400191068649, -0.02654193341732025, 0..."


In [16]:
df_train["embedp"] = df_train[[f"emb_{i}" for i in range(767)]].apply(list, axis=1)
df_train.head(5)
df_test["embedding"] = df_test["metric_name"].map(df_embed.set_index(df_metric_name[0]).apply(list, axis=1).to_dict())
df_test["embedp"] = df_test[[f"emb_{i}" for i in range(767)]].apply(list, axis=1)

In [17]:
df_test.head(5)

,metric_name,prompt_response,emb_0,emb_1,emb_2,emb_3,emb_4,emb_5,emb_6,emb_7,...,emb_760,emb_761,emb_762,emb_763,emb_764,emb_765,emb_766,emb_767,embedding,embedp
0,misuse,"PROMPT: ""my organic farm is facing a severe in...",-0.046981,-0.016840,-0.030382,0.036959,0.016839,0.012843,0.002468,0.015261,...,0.009416,-0.000461,0.000936,-0.083701,-0.012255,-0.009802,-0.023075,-0.012057,"[-0.08837708085775375, -0.03150083124637604, 0...","[-0.046981435, -0.016840002, -0.03038221, 0.03..."
1,fluency_score/syntactic_fluency,"PROMPT: ""lately i've realized that i'm spendin...",-0.023843,0.016729,0.043811,0.023247,0.009288,-0.023797,0.018355,0.061764,...,-0.036337,-0.025301,0.036675,-0.011512,0.034916,0.039179,-0.001251,0.009909,"[-0.05318500101566315, -0.0026239403523504734,...","[-0.023842609, 0.016728982, 0.04381057, 0.0232..."
2,fairness,"PROMPT: ""two students are participating in a n...",0.016981,-0.045997,0.002208,0.005010,-0.046943,0.025533,-0.027447,0.031859,...,0.026431,0.013714,0.008489,-0.040994,0.002897,-0.008517,-0.079691,-0.006323,"[-0.08156897872686386, -0.007620691321790218, ...","[0.016980618, -0.045997497, 0.0022075418, 0.00..."
3,privacy_leakage,"PROMPT: ""i'm following up on the 'synthesis' c...",-0.000102,0.011604,0.010585,0.030852,-0.030756,-0.030130,-0.071343,0.017324,...,0.015124,-0.048384,0.001643,-0.054637,0.030592,-0.023613,-0.025322,0.019453,"[-0.12314088642597198, 0.00825826171785593, 0....","[-0.000102290025, 0.011603798, 0.010584968, 0...."
4,topic_drift_rate,"PROMPT: ""a company develops a mental wellness ...",-0.080142,-0.007842,0.093260,0.049535,0.006541,0.003512,-0.048643,0.042959,...,-0.030006,-0.005198,-0.068099,-0.071701,-0.006216,0.028427,0.009236,-0.080317,"[-0.08305344730615616, -0.008159289136528969, ...","[-0.08014172, -0.007842497, 0.09326041, 0.0495..."


##### vectors normalized

In [18]:
df_train["embedding"] = df_train["embedding"].apply(
    lambda x: (np.array(x) / (np.linalg.norm(x) + 1e-10)).tolist()
)
df_train["embedp"] = df_train["embedp"].apply(
    lambda x: (np.array(x) / (np.linalg.norm(x) + 1e-10)).tolist()
)
df_test["embedding"] = df_test["embedding"].apply(
    lambda x: (np.array(x) / (np.linalg.norm(x) + 1e-10)).tolist()
)
df_test["embedp"] = df_test["embedp"].apply(
    lambda x: (np.array(x) / (np.linalg.norm(x) + 1e-10)).tolist()
)
df_train["target_sim"] = (df_train["score"] ) / 10

In [19]:
df_train.head(5)

,metric_name,score,prompt_response,emb_0,emb_1,emb_2,emb_3,emb_4,emb_5,emb_6,...,emb_761,emb_762,emb_763,emb_764,emb_765,emb_766,emb_767,embedding,embedp,target_sim
0,rejection_rate,10.0,"PROMPT: ""i have a stomach ache for a while. yo...",-0.042148,-0.030489,0.090856,0.025993,-0.012668,0.051724,-0.006721,...,-0.055014,-0.003798,-0.071675,0.042370,-0.005608,-0.071843,0.044012,"[-0.03866274568605979, 0.0007846556661677024, ...","[-0.04218889654358842, -0.030518812369546358, ...",1.0
1,exaggerated_safety/overblocking_harmless_queries,10.0,"PROMPT: ""i am writing an assignment on psychol...",-0.003471,-0.037611,0.020949,0.058486,-0.058313,0.015730,-0.030010,...,-0.022690,-0.042065,-0.076644,0.039308,0.024354,-0.031449,0.027688,"[-0.07613262248721038, -0.007797185336663332, ...","[-0.003472291428896211, -0.03762502108405031, ...",1.0
2,bias_detection/religious_or_cultural_misrepres...,10.0,"PROMPT: ""in some cultures, particularly those ...",-0.038248,-0.005651,-0.020821,0.025027,-0.012414,-0.039244,-0.042507,...,-0.044201,-0.001514,-0.036844,0.020545,-0.052852,-0.065520,-0.014510,"[-0.05974845373786236, -0.011365960875551823, ...","[-0.0382523634631802, -0.005651233303948415, -...",1.0
3,confidence_agreement/misalignment_under_safety...,10.0,"PROMPT: ""given the extensive global research a...",-0.006390,0.049337,0.012888,0.016757,0.026261,-0.010643,-0.016471,...,0.001031,0.001662,-0.089606,0.027815,0.007530,-0.034645,0.036549,"[-0.09816125047844404, -0.006414661657821297, ...","[-0.006394276257661394, 0.04936971577320619, 0...",1.0
4,cultural_sensitivity,9.0,"PROMPT: ""when modern agricultural practices su...",-0.044603,-0.020059,0.016042,-0.004787,-0.002096,-0.010011,-0.001378,...,0.004091,-0.002358,-0.075731,-0.047717,-0.011064,0.009190,-0.023655,"[-0.09363399805348342, -0.026541932323939373, ...","[-0.04461550946752, -0.0200650121114264, 0.016...",0.9


In [20]:


import numpy as np
import pandas as pd

D = df_train["embedp"].apply(lambda v: 0 if v is None else len(v)).max()
print("Detected target embedding dimension =", D)


def clean_vector(v, target_dim):
    # convert to numpy
    v = np.array(v, np.float32)

    # (A) Fix NaN → replace with 0
    if np.isnan(v).any():
        v = np.nan_to_num(v, nan=0.0)

    # (B) Fix wrong dimension
    if len(v) == target_dim:
        return v

    if len(v) == target_dim - 1:  # missing 1 element
        return np.pad(v, (0,1))   # add one zero


    if len(v) > target_dim:
        return v[:target_dim]

    if len(v) < target_dim:
        return np.pad(v, (0, target_dim - len(v)))

    raise ValueError(f"Unexpected vector length {len(v)}")

# -----------------------------
df_train["embedp"]   = df_train["embedp"].apply(lambda v: clean_vector(v, D))
df_train["embedding"] = df_train["embedding"].apply(lambda v: clean_vector(v, D))

# -----------------------------
# 4. Print summary
# -----------------------------
print("\nFinal dimension check:")
print(df_train["embedp"].apply(len).value_counts())
print(df_train["embedding"].apply(len).value_counts())

print("\nNaN check after cleaning:")
print("embedp NaN present:", df_train["embedp"].apply(lambda v: np.isnan(v).any()).any())
print("embedding NaN present:", df_train["embedding"].apply(lambda v: np.isnan(v).any()).any())

print("\n>>> All embeddings cleaned, padded, and NaN-free. <<<")


Detected target embedding dimension = 767

Final dimension check:
embedp
767    5000
Name: count, dtype: int64
embedding
767    5000
Name: count, dtype: int64

NaN check after cleaning:
embedp NaN present: False
embedding NaN present: False

>>> All embeddings cleaned, padded, and NaN-free. <<<


In [ ]:


import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import numpy as np
from tqdm import tqdm
# ------------------------------------------------------------
D = len(df_train["embedp"].iloc[0])

DEVICE = "cuda"
LR = 3e-3
EPOCHS = 600
BATCH = 32
EMB_DIM = 512
USE_COS = False



class LSMDDataset(Dataset):
    def __init__(self, df):
        self.x = df["embedp"].apply(lambda v: np.array(v, np.float32)).values
        self.p = df["embedding"].apply(lambda v: np.array(v, np.float32)).values
        self.score = df["score"].astype(np.float32).values

    def __len__(self):
        return len(self.x)

    def __getitem__(self, i):
        return {
            "x": torch.tensor(self.x[i]),          # (D,)
            "p": torch.tensor(self.p[i]),          # (D,)
            "y": torch.tensor(self.score[i] / 10)  # BEST y = score/10
        }


train_ds = LSMDDataset(df_train)
loader = DataLoader(train_ds, batch_size=BATCH, shuffle=True)


#                    CNN ENCODER


class CNNEncoder(nn.Module):
    def __init__(self, in_dim=D, out_dim=EMB_DIM):
        super().__init__()

        self.conv = nn.Sequential(
            nn.Conv1d(1, 64, kernel_size=5, padding=2),
            nn.ReLU(),
            nn.MaxPool1d(2),       # D → D/2
            nn.Dropout(0.1),

            nn.Conv1d(64, 128, kernel_size=5, padding=2),
            nn.ReLU(),
            nn.MaxPool1d(2),       # D/2 → D/4
            nn.Dropout(0.1),
        )

        pooled_len = in_dim // 4

        self.flat_dim = 128 * pooled_len

        self.fc = nn.Sequential(
            nn.Linear(self.flat_dim, 512),
            nn.ReLU(),
            nn.Linear(512, out_dim),
        )

    def forward(self, x):
        x = x.unsqueeze(1)          # (B,1,D)
        x = self.conv(x)            # (B,128,D/4)
        x = x.reshape(x.size(0), -1)
        x = self.fc(x)
        return nn.functional.normalize(x, p=2, dim=1)



class ProtoProj(nn.Module):
    def __init__(self, in_dim=D, out_dim=EMB_DIM):
        super().__init__()
        self.W = nn.Linear(in_dim, out_dim, bias=False)
        nn.init.xavier_uniform_(self.W.weight)

    def forward(self, p):
        z = self.W(p)
        return nn.functional.normalize(z, p=2, dim=1)

def lsmd_loss(student, teacher, y):
    d = torch.norm(student - teacher, p=2, dim=1)

    # pure LSMD
    loss = (1 - y) * (d**2) + y * torch.exp(-d)

    if USE_COS:
        cos = nn.functional.cosine_similarity(student, teacher, dim=1)
        loss += 0.1 * (1 - cos)

    return loss.mean()

student = CNNEncoder().to(DEVICE)
proj = ProtoProj().to(DEVICE)

opt = torch.optim.Adam(
    list(student.parameters()) + list(proj.parameters()),
    lr=LR
)

print("\n>>> TRAINING LSMD + CNN (BEST VERSION) <<<\n")

for ep in range(1, EPOCHS + 1):
    total = 0
    n = 0

    student.train(); proj.train()

    for batch in tqdm(loader, desc=f"Epoch {ep}/{EPOCHS}"):
        x = batch["x"].to(DEVICE)   # (B,D)
        p = batch["p"].to(DEVICE)
        y = batch["y"].to(DEVICE)   # y = score/10

        s = student(x)
        t = proj(p)

        loss = lsmd_loss(s, t, y)

        opt.zero_grad()
        loss.backward()

        nn.utils.clip_grad_norm_(student.parameters(), 1.0)
        nn.utils.clip_grad_norm_(proj.parameters(), 1.0)

        opt.step()

        total += loss.item() * x.size(0)
        n += x.size(0)

    print(f"Epoch {ep} Loss = {total/n:.6f}")



In [ ]:
def normalize_cnn_input(x):
    while x.dim() > 3:
        x = x.squeeze(1)    # remove middle 1s

    if x.dim() == 2:
        x = x.unsqueeze(1)      # → (B,1,D)

    return x

In [ ]:

df_test["embedp"]   = df_test["embedp"].apply(lambda v: clean_vector(v, D))
df_test["embedding"] = df_test["embedding"].apply(lambda v: clean_vector(v, D))

# -----------------------------
# 4. Print summary
# -----------------------------
print("\n[Test] Final dimension check:")
print(df_test["embedp"].apply(len).value_counts())
print(df_test["embedding"].apply(len).value_counts())

print("\n[Test] NaN check after cleaning:")
print("embedp NaN present:", df_test["embedp"].apply(lambda v: np.isnan(v).any()).any())
print("embedding NaN present:", df_test["embedding"].apply(lambda v: np.isnan(v).any()).any())

print("\n>>> TEST embeddings cleaned, padded, and NaN-free. <<<")


In [ ]:
def predict(df_test, student, proj):
    preds = []
    student.eval()
    proj.eval()

    for i in range(len(df_test)):
        x = torch.tensor(df_test["embedp"].iloc[i], dtype=torch.float32).unsqueeze(0)
        p = torch.tensor(df_test["embedding"].iloc[i], dtype=torch.float32).unsqueeze(0)

        with torch.no_grad():
            s = student(x)
            t = proj(p)

            d = torch.norm(s - t, p=2).item()
            score = 10 * torch.sigmoid(torch.tensor(1*(1- d/2))).item()

        preds.append(score)

    return preds



In [ ]:
df_test["score"] = predict(df_test, student, proj)
print("Prediction completed!")

In [ ]:
submission = pd.DataFrame({
    "ID": range(1, len(df_test)+1),
    "score": df_test["score"]
})
submission.to_csv("submission.csv", index=False)


In [ ]:
import os
os.getcwd()

In [ ]:

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import json
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", DEVICE)



df_train = pd.read_csv('/content/response_prompt_embed_train.csv')
df_test  = pd.read_csv('/content/response_prompt_embed_test.csv')

embed = np.load('/content/metric_name_embeddings.npy')
with open('/content/metric_names.json','r') as f:
    metric_names = json.load(f)
df_metric_name = pd.DataFrame(metric_names)

df_embed = pd.DataFrame(embed)

metric_dict = df_embed.set_index(df_metric_name[0]).apply(list, axis=1).to_dict()
df_train["embedding"] = df_train["metric_name"].map(metric_dict)
df_test["embedding"]  = df_test["metric_name"].map(metric_dict)

# Prompt-response embeddings
df_train["embedp"] = df_train[[f"emb_{i}" for i in range(767)]].apply(list, axis=1)
df_test["embedp"]  = df_test[[f"emb_{i}" for i in range(767)]].apply(list, axis=1)


def clean_vector(v, target_dim):
    v = np.array(v, np.float32)
    if np.isnan(v).any():
        v = np.nan_to_num(v, nan=0.0)
    if len(v) == target_dim:
        return v / (np.linalg.norm(v) + 1e-10)
    if len(v) < target_dim:
        v = np.pad(v, (0, target_dim - len(v)))
    if len(v) > target_dim:
        v = v[:target_dim]
    return v / (np.linalg.norm(v) + 1e-10)

D = df_train["embedp"].apply(lambda v: len(v)).max()

for col in ["embedp", "embedding"]:
    df_train[col] = df_train[col].apply(lambda v: clean_vector(v, D))
    df_test[col]  = df_test[col].apply(lambda v: clean_vector(v, D))

# Target scaling
df_train["score_norm"] = df_train["score"] / 10.0

class LSMDDataset(Dataset):
    def __init__(self, df):
        self.x = df["embedp"].apply(lambda v: np.array(v, np.float32)).values
        self.p = df["embedding"].apply(lambda v: np.array(v, np.float32)).values
        self.y = df["score_norm"].values.astype(np.float32)

    def __len__(self):
        return len(self.x)

    def __getitem__(self, i):
        return {
            "x": torch.tensor(self.x[i]),
            "p": torch.tensor(self.p[i]),
            "y": torch.tensor(self.y[i])
        }

EMB_DIM = 256
USE_COS = True

class CNNEncoder(nn.Module):
    def __init__(self, in_dim, out_dim=EMB_DIM):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv1d(1, 64, kernel_size=5, padding=2),
            nn.ReLU(),
            nn.MaxPool1d(2),
            nn.Dropout(0.1),
            nn.Conv1d(64, 128, kernel_size=5, padding=2),
            nn.ReLU(),
            nn.MaxPool1d(2),
            nn.Dropout(0.1)
        )
        pooled_len = in_dim // 4
        self.fc = nn.Sequential(
            nn.Linear(128 * pooled_len, 512),
            nn.ReLU(),
            nn.Linear(512, out_dim)
        )

    def forward(self, x):
        x = x.unsqueeze(1)
        x = self.conv(x)
        x = x.reshape(x.size(0), -1)
        return nn.functional.normalize(self.fc(x), p=2, dim=1)

class ProtoProj(nn.Module):
    def __init__(self, in_dim, out_dim=EMB_DIM):
        super().__init__()
        self.W = nn.Linear(in_dim, out_dim, bias=False)
        nn.init.xavier_uniform_(self.W.weight)

    def forward(self, p):
        z = self.W(p)
        return nn.functional.normalize(z, p=2, dim=1)

# LSMD loss
def lsmd_loss(student, teacher, y):
    d = torch.norm(student - teacher, p=2, dim=1)
    loss = (1 - y) * (d**2) + y * torch.exp(-d)
    if USE_COS:
        cos_sim = nn.functional.cosine_similarity(student, teacher, dim=1)
        loss += 0.05 * (1 - cos_sim)
    return loss.mean()

def train_model(df_train, input_dim, epochs=200, batch_size=32, lr=1e-3):
    dataset = LSMDDataset(df_train)
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    student = CNNEncoder(input_dim).to(DEVICE)
    teacher = ProtoProj(input_dim).to(DEVICE)
    optimizer = torch.optim.Adam(list(student.parameters()) + list(teacher.parameters()), lr=lr)

    for ep in range(1, epochs+1):
        student.train(); teacher.train()
        total_loss = 0; n = 0
        for batch in loader:
            x = batch["x"].to(DEVICE)
            p = batch["p"].to(DEVICE)
            y = batch["y"].to(DEVICE)

            s = student(x)
            t = teacher(p)

            loss = lsmd_loss(s, t, y)

            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(student.parameters(), 1.0)
            torch.nn.utils.clip_grad_norm_(teacher.parameters(), 1.0)
            optimizer.step()

            total_loss += loss.item() * x.size(0)
            n += x.size(0)

        if ep % 10 == 0 or ep == 1:
            print(f"Epoch {ep}/{epochs} | Loss: {total_loss/n:.6f}")

    return student, teacher


def predict(df_test, student, teacher):
    student.eval(); teacher.eval()
    preds = []
    for i in range(len(df_test)):
        x = torch.tensor(df_test["embedp"].iloc[i], dtype=torch.float32).unsqueeze(0).to(DEVICE)
        p = torch.tensor(df_test["embedding"].iloc[i], dtype=torch.float32).unsqueeze(0).to(DEVICE)
        with torch.no_grad():
            s = student(x)
            t = teacher(p)
            d = torch.norm(s - t, p=2).item()
            score = 10 * torch.sigmoid(1 - d/2)
            preds.append(score)
    df_test["score"] = preds
    return df_test


D = len(df_train["embedp"].iloc[0])
student_model, teacher_model = train_model(df_train, input_dim=D, epochs=200)
df_test = predict(df_test, student_model, teacher_model)

submission = pd.DataFrame({
    "ID": range(1, len(df_test)+1),
    "score": df_test["score"]
})
submission.to_csv("submission.csv", index=False)
print("Submission file created!")


In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import json
from sklearn.model_selection import train_test_split

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", DEVICE)

df_train = pd.read_csv('/content/response_prompt_embed_train.csv')
df_test  = pd.read_csv('/content/response_prompt_embed_test.csv')

embed = np.load('/content/metric_name_embeddings.npy')
with open('/content/metric_names.json','r') as f:
    metric_names = json.load(f)
df_metric_name = pd.DataFrame(metric_names)

df_embed = pd.DataFrame(embed)

metric_dict = df_embed.set_index(df_metric_name[0]).apply(list, axis=1).to_dict()
df_train["embedding"] = df_train["metric_name"].map(metric_dict)
df_test["embedding"]  = df_test["metric_name"].map(metric_dict)

df_train["embedp"] = df_train[[f"emb_{i}" for i in range(767)]].apply(list, axis=1)
df_test["embedp"]  = df_test[[f"emb_{i}" for i in range(767)]].apply(list, axis=1)

def clean_vector(v, target_dim):
    v = np.array(v, np.float32)
    if np.isnan(v).any():
        v = np.nan_to_num(v, nan=0.0)
    if len(v) == target_dim:
        return v / (np.linalg.norm(v) + 1e-10)
    if len(v) < target_dim:
        v = np.pad(v, (0, target_dim - len(v)))
    if len(v) > target_dim:
        v = v[:target_dim]
    return v / (np.linalg.norm(v) + 1e-10)

D = df_train["embedp"].apply(lambda v: len(v)).max()

for col in ["embedp", "embedding"]:
    df_train[col] = df_train[col].apply(lambda v: clean_vector(v, D))
    df_test[col]  = df_test[col].apply(lambda v: clean_vector(v, D))

df_train["score_norm"] = df_train["score"] / 10.0

class LSMDDataset(Dataset):
    def __init__(self, df):
        self.x = df["embedp"].apply(lambda v: np.array(v, np.float32)).values
        self.p = df["embedding"].apply(lambda v: np.array(v, np.float32)).values
        self.y = df["score_norm"].values.astype(np.float32)

    def __len__(self):
        return len(self.x)

    def __getitem__(self, i):
        return {
            "x": torch.tensor(self.x[i]),
            "p": torch.tensor(self.p[i]),
            "y": torch.tensor(self.y[i])
        }

EMB_DIM = 256
USE_COS = True

class CNNEncoder(nn.Module):
    def __init__(self, in_dim, out_dim=EMB_DIM):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv1d(1, 64, kernel_size=5, padding=2),
            nn.ReLU(),
            nn.MaxPool1d(2),
            nn.Dropout(0.1),
            nn.Conv1d(64, 128, kernel_size=5, padding=2),
            nn.ReLU(),
            nn.MaxPool1d(2),
            nn.Dropout(0.1)
        )
        pooled_len = in_dim // 4
        self.fc = nn.Sequential(
            nn.Linear(128 * pooled_len, 512),
            nn.ReLU(),
            nn.Linear(512, out_dim)
        )

    def forward(self, x):
        x = x.unsqueeze(1)
        x = self.conv(x)
        x = x.reshape(x.size(0), -1)
        return nn.functional.normalize(self.fc(x), p=2, dim=1)

class ProtoProj(nn.Module):
    def __init__(self, in_dim, out_dim=EMB_DIM):
        super().__init__()
        self.W = nn.Linear(in_dim, out_dim, bias=False)
        nn.init.xavier_uniform_(self.W.weight)

    def forward(self, p):
        z = self.W(p)
        return nn.functional.normalize(z, p=2, dim=1)

def lsmd_loss(student, teacher, y):
    d = torch.norm(student - teacher, p=2, dim=1)
    loss = (1 - y) * (d**2) + y * torch.exp(-d)
    if USE_COS:
        cos_sim = nn.functional.cosine_similarity(student, teacher, dim=1)
        loss += 0.05 * (1 - cos_sim)
    return loss.mean()

def train_model(df_train, input_dim, epochs=200, batch_size=32, lr=1e-3):

    df_tr, df_val = train_test_split(df_train, test_size=0.1, random_state=42)

    train_loader = DataLoader(LSMDDataset(df_tr), batch_size=batch_size, shuffle=True)
    val_loader   = DataLoader(LSMDDataset(df_val), batch_size=batch_size, shuffle=False)

    student = CNNEncoder(input_dim).to(DEVICE)
    teacher = ProtoProj(input_dim).to(DEVICE)
    optimizer = torch.optim.Adam(list(student.parameters()) + list(teacher.parameters()), lr=lr)

    for ep in range(1, epochs+1):
        student.train(); teacher.train()

        total_loss = 0; n = 0
        for batch in train_loader:
            x = batch["x"].to(DEVICE)
            p = batch["p"].to(DEVICE)
            y = batch["y"].to(DEVICE)

            s = student(x)
            t = teacher(p)
            loss = lsmd_loss(s, t, y)

            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(student.parameters(), 1.0)
            torch.nn.utils.clip_grad_norm_(teacher.parameters(), 1.0)
            optimizer.step()

            total_loss += loss.item() * x.size(0)
            n += x.size(0)

        # -----------------------------
        # VALIDATION LOSS
        # -----------------------------
        student.eval(); teacher.eval()
        val_loss = 0; vn = 0

        with torch.no_grad():
            for batch in val_loader:
                x = batch["x"].to(DEVICE)
                p = batch["p"].to(DEVICE)
                y = batch["y"].to(DEVICE)

                s = student(x)
                t = teacher(p)
                loss = lsmd_loss(s, t, y)

                val_loss += loss.item() * x.size(0)
                vn += x.size(0)

        if ep % 10 == 0 or ep == 1:
            print(f"Epoch {ep}/{epochs} | Train Loss: {total_loss/n:.6f} | Val Loss: {val_loss/vn:.6f}")

    return student, teacher


def predict(df_test, student, teacher):
    student.eval(); teacher.eval()
    preds = []
    for i in range(len(df_test)):
        x = torch.tensor(df_test["embedp"].iloc[i], dtype=torch.float32).unsqueeze(0).to(DEVICE)
        p = torch.tensor(df_test["embedding"].iloc[i], dtype=torch.float32).unsqueeze(0).to(DEVICE)
        with torch.no_grad():
            s = student(x)
            t = teacher(p)
            d = torch.norm(s - t, p=2).item()
            score = 10 * torch.sigmoid(1 - d/2)
            preds.append(score)
    df_test["score"] = preds
    return df_test


D = len(df_train["embedp"].iloc[0])
student_model, teacher_model = train_model(df_train, input_dim=D, epochs=200)

df_test = predict(df_test, student_model, teacher_model)

submission = pd.DataFrame({
    "ID": range(1, len(df_test)+1),
    "score": df_test["score"]
})
submission.to_csv("submission.csv", index=False)
print("Submission file created!")
